Convert worlds into compressed pytorch files. 

Simply place your worlds into worlds/minecraft and run the below code.

Compressed worlds with chunks stored as tensors will then appear in the worlds/compressed.

In [1]:
import os
from pathlib import Path

from data_prep.world_parsing.block_id_mapping import BlockIDMapper
from data_prep.world_parsing.convert_to_tensors import TensorConverter
from data_prep.world_parsing.torch_world import compress_world
from data_prep.world_parsing.utils import get_dir_size

In [2]:
NUM_CORES = 8

In [3]:
dpp = Path('/Users/ben/Projects/MinecraftDiffusion/code/data_prep')
mapper = BlockIDMapper(Path(dpp / "fixtures/all_blocks.tsv"))
tensor_converter = TensorConverter(mapper, num_cores=NUM_CORES)

In [4]:
worlds_dir      = dpp / 'pipelines/torchify/worlds'
raw_dir         = worlds_dir / 'minecraft'
tensor_dir      = worlds_dir / 'pytorch'
compressed_dir  = worlds_dir / 'compressed'

In [5]:
raw_worlds = set(os.listdir(raw_dir))
# Trim '.tar.gz' extension.
compressed_worlds = set([p[:-7] for p in os.listdir(compressed_dir)])
to_convert = raw_worlds - compressed_worlds

if '.DS_Store' in to_convert:
    to_convert.discard('.DS_Store')

print(' === Converting ===')
for filename in to_convert:
    size_str = get_dir_size(raw_dir / filename)
    print(f'{filename:30} {size_str:>10}')

 === Converting ===
massive_plains                  703.33 MB


In [6]:
for world in to_convert:
    print(f'Now converting {world}')
    world_path = raw_dir / world
    tensor_world_path = tensor_dir / world
    tensor_world_path.mkdir(exist_ok=True)
    tensor_converter.convert_world_to_torch(world_path, tensor_world_path, overwrite=False, version='vectorized')
    compress_world(tensor_world_path, compressed_dir)
    os.system(f'rm -rf {str(tensor_world_path)}')

Now converting massive_plains


100%|██████████| 81/81 [01:00<00:00,  1.35it/s]
